In [1]:
from typing import List, Dict
import os
from pathlib import Path
import datetime as dt

import matplotlib.pyplot as plt
import mplfinance as mpf
import pandas as pd
import numpy as np
import math


# Definations.

# Package path.
PACKAGE_PATH: Path = Path(os.path.abspath(''))

# Timezone.
tz_beijing: dt.timezone = dt.timezone(dt.timedelta(hours=8))
tz_trading: dt.timezone = dt.timezone(dt.timedelta(hours=12))

tz_delta: dt.timedelta = dt.timedelta(hours=4)


def filter_non_trading_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Drop data in non-trading time.
    """
    if df.index.inferred_type == 'datetime64':
        return df[
            (
                ((df.index.time >= dt.datetime.strptime('00:59', '%H:%M').time()) & (df.index.time <= dt.datetime.strptime('05:00', '%H:%M').time())) |
                ((df.index.time >= dt.datetime.strptime('12:59', '%H:%M').time()) & (df.index.time <= dt.datetime.strptime('14:15', '%H:%M').time())) |
                ((df.index.time >= dt.datetime.strptime('14:30', '%H:%M').time()) & (df.index.time <= dt.datetime.strptime('15:30', '%H:%M').time())) |
                ((df.index.time >= dt.datetime.strptime('17:30', '%H:%M').time()) & (df.index.time <= dt.datetime.strptime('19:00', '%H:%M').time()))
            )
        ]
    else:
        return df[
            (
                ((df.datetime.dt.time >= dt.datetime.strptime('00:59', '%H:%M').time()) & (df.datetime.dt.time <= dt.datetime.strptime('05:00', '%H:%M').time())) |
                ((df.datetime.dt.time >= dt.datetime.strptime('12:59', '%H:%M').time()) & (df.datetime.dt.time <= dt.datetime.strptime('14:15', '%H:%M').time())) |
                ((df.datetime.dt.time >= dt.datetime.strptime('14:30', '%H:%M').time()) & (df.datetime.dt.time <= dt.datetime.strptime('15:30', '%H:%M').time())) |
                ((df.datetime.dt.time >= dt.datetime.strptime('17:30', '%H:%M').time()) & (df.datetime.dt.time <= dt.datetime.strptime('19:00', '%H:%M').time()))
            )
        ]

def pbx(df: pd.DataFrame) -> pd.DataFrame:
    """
    Add PBX indicator.
    """
    df['PBX4'] = (df['close'].ewm(span=4, adjust=False).mean()+df['close'].rolling(4*2).mean()+df['close'].rolling(4*4).mean())/3
    df['PBX6'] = (df['close'].ewm(span=6, adjust=False).mean()+df['close'].rolling(6*2).mean()+df['close'].rolling(6*4).mean())/3
    df['PBX24'] = (df['close'].ewm(span=24, adjust=False).mean()+df['close'].rolling(24*2).mean()+df['close'].rolling(24*4).mean())/3
    return df

In [2]:
# data file.
tick_path: Path = PACKAGE_PATH.joinpath('SHFE.al2111_Tick.csv')

In [3]:
# Load csv to DataFrame.
df_origin: pd.DataFrame = pd.read_csv(tick_path, parse_dates=['datetime'], index_col=['datetime'])

# Change timezone.
df_trading = df_origin
df_trading.index = df_origin.index + dt.timedelta(hours=4)

# Drop non-trading time data.
df_trading = filter_non_trading_data(df_trading)

In [4]:
df_1min = df_trading['last_price'].resample('1MIN').ohlc()
df_1min = pbx(df_1min)